In [5]:
from scipy.optimize import minimize_scalar
from scipy.optimize import fsolve
from scipy import optimize
import numpy as np
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator


datapath = Path.cwd().parent / "data" / "eval" / "2-eval_cls_perturb.csv"
data = pd.read_csv(datapath)

"""
preprocessing
"""

data = data.replace([np.inf, -np.inf], np.nan).dropna()
keys = ['opacity', 'density', 'perturb', 'epsilon', 'img_id']
data = data.drop_duplicates(subset=keys)

# difference in accuracy between adversarial and benign examples - we want the performance to decrease relative to what it initially was (lower is better)
data['adv_acc'] = ((data['advx_acc1'] + data['advx_acc5']) / 2) - ((data['x_acc1'] + data['x_acc5']) / 2) + 1
# perceptual quality, approximates human perception (higher is better)
data['quality'] = (data['cosine_sim'] + data['psnr'] / 100 + data['ssim']) / 3

# group - get mean of just the 2 inferred metrics
# opacity,density,perturb,epsilon
data = data.groupby(['opacity', 'density', 'perturb', 'epsilon']).agg({
    'adv_acc': 'mean',
    'quality': 'mean',
}).reset_index()

data = data.replace([np.inf, -np.inf], np.nan).dropna()
if np.isinf(data['adv_acc']).any() or np.isinf(data['quality']).any():
    print("\nwarning: Infinity values still present in the data")

# """
# scatter plot
# """

# markers = {'diamond': 'D'}
# colors = {0: 'lightblue', 64: 'skyblue', 128: 'deepskyblue', 192: 'dodgerblue', 255: 'blue'}

# plt.figure(figsize=(15, 15))

# for mask in markers:
#     for opacity in colors:
#         mask_opacity_data = data[(data['opacity'] == opacity)]
#         plt.scatter(mask_opacity_data['adv_acc'], mask_opacity_data['quality'], 
#                     marker=markers[mask], c=colors[opacity], s=mask_opacity_data['density']*2,
#                     label=f'{mask}, opacity={opacity}')

# """
# max y value
# """

# x = data['adv_acc']
# y = data['quality']

# max_coords = []
# for x_elem in x:
#     y_elem = y[x == x_elem]
#     max_y = y_elem.max()
#     max_coords.append((x_elem, max_y))
#     plt.scatter(x_elem, max_y, marker='o', facecolors='none', edgecolors='gray', s=3)

# max_coords = sorted(max_coords, key=lambda x: x[0])
# max_x, max_y = zip(*max_coords)
# plt.plot(max_x, max_y, color='gray', linestyle='--')

# max_coords_df = []
# for x_elem, y_elem in max_coords:
#     max_coords_df.append(data[(data['adv_acc'] == x_elem) & (data['quality'] == y_elem)].iloc[0])
# max_coords_df = pd.DataFrame(max_coords_df)
# max_coords_df = max_coords_df.drop_duplicates(subset=['mask', 'opacity', 'density']).reset_index(drop=True)
# max_coords_df['final_score'] = -max_coords_df['adv_acc'] + max_coords_df['quality']

# # approximate with a polynomial regression
# x = np.array(max_x).reshape(-1, 1)
# y = np.array(max_y)
# maxy_degree = 1
# polyreg = make_pipeline(PolynomialFeatures(maxy_degree), LinearRegression())
# polyreg.fit(x, y)
# x_smooth = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
# y_smooth = polyreg.predict(x_smooth)
# plt.plot(x_smooth, y_smooth, color='red', linestyle='--')

# print("\nAll above the regression line:")
# above = [data[(data['adv_acc'] == x_elem) & (data['quality'] == y_elem)].iloc[0] for x_elem, y_elem in max_coords if y_elem > polyreg.predict([[x_elem]])]
# above = pd.DataFrame(above)
# above = above.drop_duplicates(subset=['mask', 'opacity', 'density']).reset_index(drop=True)
# above['score'] = -above['adv_acc'] + above['quality']
# display(HTML(above.to_html()))

# # print("\nMax Quality per Accuracy (sorted by x coordinate):")
# # max_coords_df = max_coords_df.sort_values(by='adv_acc')
# # display(HTML(max_coords_df.to_html()))

# max_coords_df = max_coords_df.sort_values(by='final_score', ascending=False)
# print("\nMax Quality per Accuracy (sorted by final_score):")
# display(HTML(max_coords_df.to_html()))

# """
# polynomial regression
# """

# x = data['adv_acc'].values.reshape(-1, 1)
# y = data['quality'].values

# total_degree = 1
# polyreg = make_pipeline(PolynomialFeatures(total_degree), LinearRegression())
# polyreg.fit(x, y)

# x_smooth = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
# y_smooth = polyreg.predict(x_smooth)

# plt.plot(x_smooth, y_smooth, color='gray')

# """
# plot
# """

# plt.xlabel('Adversarial Accuracy (lower is better)')
# plt.ylabel('Perceptual Quality (higher is better)')
# plt.title('Trade-off between Adversarial Accuracy and Perceptual Quality')

# # legend for markers
# legend_elements = [plt.Line2D([0], [0], marker='D', color='w', label='Diamond', markerfacecolor='gray', markersize=10)]

# legend_elements.append(plt.Line2D([0], [0], marker='None', color='w', label=''))

# # legend for opacity
# for opacity, color in colors.items():
#     legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=f'Opacity {opacity}', markerfacecolor=color, markersize=10))

# legend_elements.append(plt.Line2D([0], [0], marker='None', color='w', label=''))

# # legend for density
# legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label='Low Density', markerfacecolor='gray', markersize=5))
# legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label='High Density', markerfacecolor='gray', markersize=10))

# legend_elements.append(plt.Line2D([0], [0], marker='None', color='w', label=''))

# # legend for max y value
# legend_elements.append(plt.Line2D([0], [0], color='gray', linestyle='--', label=f'Max quality per accuracy'))
# legend_elements.append(plt.Line2D([0], [0], color='red', linestyle='--', label=f'Poly Regression for max quality (degree={maxy_degree})'))

# # legend for polynomial regression
# legend_elements.append(plt.Line2D([0], [0], color='gray', label=f'Poly Regression for full dataset (degree={total_degree})'))

# plt.legend(handles=legend_elements, title='Legend', loc='center left', bbox_to_anchor=(1, 0.5))

# # add grid
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.minorticks_on()
# plt.gca().xaxis.set_minor_locator(AutoMinorLocator())
# plt.gca().yaxis.set_minor_locator(AutoMinorLocator())

# plt.tight_layout()
# plt.show()
